**Zadanie 1**  
Zbiór (niemały) logów z różnych systemów do wykorzystania w badaniach. Zwróć uwagę na licencję i cytowanie w razie wykorzystania zbioru.

Link: https://github.com/logpai/loghub

W przykładzie poniżej zostanie wykorzystany log serwera Apache ze strony wymienionej wyżej.
Link do pliku sample: https://github.com/logpai/loghub/blob/master/Apache/Apache_2k.log

**Zastanów się i spróbuj przerobić to rozwiązanie tak, aby możliwe było użycie wywołań dask delayed w celu zrównoleglenia części funkcji, np. parsowanie danych w celu pobrania wartości kolumn niezależnie (tylko jednej na raz). 
Dane końcowe możesz zapisać do dask DataFrame, a następnie do plików parquet.**

Aby zyskać równie jakieś porównanie między wersją sekwencyjną a zrównolegloną, dodaj odpowiednio dużo danych ze zbiorów podlinkowanych powyżej oraz zmierz czas obu rozwiązań.

W procesie poszukiwania dobrego rozwiązania z użyciem dask delayed pomocny może być poradnik z dobrymi praktykami dla tej części biblioteki dask: https://docs.dask.org/en/latest/delayed-best-practices.html

In [9]:
from dask.distributed import Client

client = Client(n_workers=4)
client

D:\Program\anaconda3\envs\aaaaaaa\Lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 58278 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:58278/status,
Dashboard: http://127.0.0.1:58278/status,Workers: 4
Total threads: 12,Total memory: 15.78 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:58279,Workers: 4
Dashboard: http://127.0.0.1:58278/status,Total threads: 12
Started: Just now,Total memory: 15.78 GiB
Comm: tcp://127.0.0.1:58298,Total threads: 3
Dashboard: http://127.0.0.1:58301/status,Memory: 3.95 GiB
Nanny: tcp://127.0.0.1:58282,


In [10]:
# kwestia wczytania plików zostanie tu pominięta

from datetime import datetime
import time
import os
import pandas as pd


def read_lines(file_name, file_path='data'):
    with open(os.path.join(file_path, file_name), 'r') as f:
        return f.readlines()


def parse(inp: str):
    record = {}

    date_start = inp.find('[') + 1
    date_end = inp.find(']')
    date_s = slice(date_start, date_end)

    level_start = inp.find('[', date_end) + 1
    level_end = inp.find(']', level_start)
    level_s = slice(level_start, level_end)

    client_start = inp.find('[', level_end)
    client_end = inp.find(']', client_start)

    record["date"] = inp[date_s]
    record["level"] = inp[level_s]
    record["client"] = "" if client_start == -1 else inp[client_start + 8: client_end]
    record["message"] = inp[client_end + 2:] if record["client"] else inp[level_end + 2:]

    return record


def convert_date(rec):
    rec["date"] = datetime.strptime(rec["date"], "%a %b %d %H:%M:%S %Y")

    return rec


# przetworzenie loga
start = datetime.now()
output = []
lines = read_lines("Apache_2k.log")

for line in lines:
    record = parse(line)
    record = convert_date(record)
    output.append(list(record.values()))

df = pd.DataFrame(output, columns=["date", "level", "client", "message"])
df.head()
df.to_parquet(os.path.join('data','logs1.parquet'), engine='fastparquet')
print(f"time: {datetime.now() - start}")

time: 0:00:00.070000


In [11]:
import dask
import os
import pandas as pd
import dask.dataframe as dd
import fastparquet


def read_lines(file_name, file_path='data'):
    with open(os.path.join(file_path, file_name), 'r') as f:
        return f.readlines()


def load_column(column_name, data):
    column = pd.DataFrame(columns=[column_name])
    for i, l in enumerate(data):
        if column_name == 'date':
            date_start = l.find('[')
            date_end = l.find(']')
            date_s = slice(date_start + 1, date_end)
            column.loc[len(column)] = convert_date_column(l[date_s])
        elif column_name == 'level':
            first_brace = l.find('[')
            level_start = l.find('[', first_brace + 1)
            level_end = l.find(']', level_start)
            level_s = slice(level_start + 1, level_end)
            column.loc[len(column)] = l[level_s]
        elif column_name == 'client':
            brace = -1
            for i in range(2):
                brace = l.find('[', brace + 1)
            client_start = l.find('[', brace + 1)
            if client_start != -1:
                client_end = l.find(']', client_start)
                client_s = slice(client_start, client_end)
                column.loc[len(column)] = l[client_s]
            else:
                column.loc[len(column)] = None
        else:
            last_brace = l.rfind(']')
            print(last_brace)
            message = l[last_brace + 2:]
            column.loc[len(column)] = message
    # return column
    return dd.from_pandas(column, npartitions=1)


def convert_date_column(rec):
    return datetime.strptime(rec, "%a %b %d %H:%M:%S %Y")


def concat_columns(*columns):
    return dd.concat([*columns], axis=1).compute()


start = datetime.now()
lines = read_lines("Apache_2k.log")

date = dask.delayed(load_column)('date', lines)
level = dask.delayed(load_column)('level', lines)
client = dask.delayed(load_column)('client', lines)
message = dask.delayed(load_column)('message', lines)

ddf = concat_columns(date.compute(), level.compute(), client.compute(), message.compute())

ddf.to_parquet(os.path.join('data',"logs2.parquet"), engine="fastparquet")
print(f"time: {datetime.now() - start}")

time: 0:00:06.433039


**Zadanie 2**  
Wykorzystując przykłady zaprezentowane w labie wykonaj na danych `people` (możesz zmniejszyć lub zwiększyć ich wolumen w zależności od potrzeb) operację z użyciem Dask bag, 
która polegać będzie na przetworzeniu wszystkich plików i zapisaniu do plików o nazwie `expired_{partition}.json` rekordów, których ważność karty kredytowej wygasła (jest to wartość w formacie miesiąc/rok). 
Zapisując ustaw finalną liczbę plików na 10 jeżeli była inna. Możesz to zrobić poprzez zmianę ilość partycji dask bag (patrz link do API na początku laba).

In [12]:
import dask.bag as db
import dask.dataframe as dd
import json


def is_card_expired(record):
    expiration_date = record['credit-card']['expiration-date']
    month, year = map(int, expiration_date.split('/'))
    # Dodaj 2000 do roku, aby uzyskać pełny rok
    full_year = 2000 + year
    expiration = datetime(full_year, month, 1)
    return datetime.now() > expiration


# ponownie wykorzystujemy metodę map (tym razem mapujemy json.loads) na każdym pliku w folderze
b = db.read_text(os.path.join('data', '*.json')).map(json.loads)
# b = b.repartition(10)

In [13]:
# policzmy teraz ile jest osób pełnoletnich
expired = b.filter(is_card_expired)
expired = expired.repartition(10)

In [ ]:
expired.map(json.dumps).to_textfiles(os.path.join('data','expired', f'expired_*.json'))

**Zadanie 3**  
Wybierając z danych `people` dane tylko osób dorosłych (zaprezentowane w przykładach w tym labie) przechowaj je w obiekcie typu `bag`, a następnie zapisz je do dask dataframe za pomocą metody `to_dataframe` (pamiętaj o tym jaka jest aktualna struktura pojedynczego rekordu), a następnie zapisz do jednego pliku w formacie parquet.

In [15]:
count_adults = b.filter(lambda record: record['age'] >= 18).repartition(1)
df = count_adults.to_dataframe().compute()

In [16]:
df.to_parquet(os.path.join('data', "adults.parquet"), engine='fastparquet')